> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [1]:
### Escreva sua resposta aqui
import pandas as pd

file_path = 'salaries.csv'
salaries_df = pd.read_csv(file_path)
print(salaries_df.head())


  FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE DESIGNATION   AGE  \
0     TOMASA       ARMEN   F   5-18-2014   01-07-2016     Analyst  21.0   
1      ANNIE         NaN   F         NaN   01-07-2016   Associate   NaN   
2      OLIVE        ANCY   F   7-28-2014   01-07-2016     Analyst  21.0   
3     CHERRY     AQUILAR   F  04-03-2013   01-07-2016     Analyst  22.0   
4       LEON  ABOULAHOUD   M  11-20-2014   01-07-2016     Analyst   NaN   

   SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  PAST EXP  
0   44570     Finance         24.0               6.0      2.0         0  
1   89207         Web          NaN              13.0      NaN         7  
2   40955     Finance         23.0               7.0      3.0         0  
3   45550          IT         22.0               8.0      3.0         0  
4   43161  Operations         27.0               3.0      NaN         3  


#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [5]:
from sqlalchemy import create_engine, Column, Integer, String, Float, Enum, Date
from sqlalchemy.orm import declarative_base, sessionmaker
import enum

# Base do SQLAlchemy
Base = declarative_base()

# enums
class SexEnum(enum.Enum):
    M = 'M'
    F = 'F'

class DesignationEnum(enum.Enum):
    Analyst = 'Analyst'
    Associate = 'Associate'
    Manager = 'Manager'
    SeniorAnalyst = 'Senior Analyst'

class UnitEnum(enum.Enum):
    Finance = 'Finance'
    Web = 'Web'
    IT = 'IT'
    Operations = 'Operations'
    Marketing = 'Marketing'

# Definindo a classe para o ORM
class Salary(Base):
    __tablename__ = 'salaries'
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String, nullable=False)
    last_name = Column(String, nullable=True)
    sex = Column(Enum(SexEnum), nullable=True)
    doj = Column(Date, nullable=True)
    current_date = Column(Date, nullable=True)
    designation = Column(Enum(DesignationEnum), nullable=True)
    age = Column(Float, nullable=True)
    salary = Column(Float, nullable=False)
    unit = Column(Enum(UnitEnum), nullable=True)
    leaves_used = Column(Float, nullable=True)
    leaves_remaining = Column(Float, nullable=True)
    ratings = Column(Float, nullable=True)
    past_exp = Column(Float, nullable=True)

#SQLite e configurando a sessão
engine = create_engine('sqlite:///salaries.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

print("Banco de dados e tabelas criados com sucesso!")


Banco de dados e tabelas criados com sucesso!


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [7]:
### Escreva sua resposta aqui
engine = create_engine('sqlite:///salarios.db')
print("Conexão com o banco de dados 'salarios.db' estabelecida com sucesso!")


Conexão com o banco de dados 'salarios.db' estabelecida com sucesso!


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [8]:
### Escreva sua resposta aqui
Base.metadata.create_all(engine)

print("Tabelas criadas com sucesso no banco 'salarios.db'!")

Tabelas criadas com sucesso no banco 'salarios.db'!


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [12]:
# Renomear as colunas do DataFrame para corresponder ao esquema do banco
salaries_df.columns = [
    'first_name', 'last_name', 'sex', 'doj', 'current_date', 
    'designation', 'age', 'salary', 'unit', 
    'leaves_used', 'leaves_remaining', 'ratings', 'past_exp'
]

# Inserir os dados do DataFrame na tabela 'salaries'
salaries_df.to_sql('salaries', engine, if_exists='append', index=False)

print("Dados inseridos com sucesso no banco 'salarios.db'!")


Dados inseridos com sucesso no banco 'salarios.db'!


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [13]:
from sqlalchemy import text

# Criar a conexão
connection = engine.connect()

# Escrever e executar a query SQL
query = text("""
    SELECT designation, 
           MIN(salary / 12) AS min_monthly_salary,
           MAX(salary / 12) AS max_monthly_salary,
           AVG(salary / 12) AS avg_monthly_salary
    FROM salaries
    GROUP BY designation
""")

# Executar a query e obter os resultados
result = connection.execute(query)

# Exibir os resultados
for row in result:
    print(row)

# Fechar a conexão
connection.close()


('Analyst', 3333.4166666666665, 4165.0, 3751.675987685993)
('Associate', 5846.166666666667, 8300.25, 7266.915094339623)
('Director', 17832.25, 32342.666666666668, 23914.265625)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134832)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129032)


In [14]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
import pandas as pd

# Escrever a query SQL
query = """
    SELECT designation, 
           MIN(salary / 12) AS min_monthly_salary,
           MAX(salary / 12) AS max_monthly_salary,
           AVG(salary / 12) AS avg_monthly_salary
    FROM salaries
    GROUP BY designation
"""

# Executar a query usando Pandas
df = pd.read_sql_query(query, engine)

# Exibir o DataFrame
print(df)


      designation  min_monthly_salary  max_monthly_salary  avg_monthly_salary
0         Analyst         3333.416667         4165.000000         3751.675988
1       Associate         5846.166667         8300.250000         7266.915094
2        Director        17832.250000        32342.666667        23914.265625
3         Manager         8343.666667        12407.500000        10522.716049
4  Senior Analyst         4170.333333         5830.500000         4991.778792
5  Senior Manager        12614.416667        16631.416667        14888.689516


In [16]:
from sqlalchemy.orm import Session
from sqlalchemy import func, cast, String

# Criar a sessão
session = Session(engine)

# Construir a query ORM
query = session.query(
    cast(Salary.designation, String).label("designation"),  # Ignorar o Enum
    func.min(Salary.salary / 12).label("min_monthly_salary"),
    func.max(Salary.salary / 12).label("max_monthly_salary"),
    func.avg(Salary.salary / 12).label("avg_monthly_salary")
).group_by(Salary.designation)

# Executar a query e exibir os resultados
for row in query.all():
    print(row)

# Fechar a sessão
session.close()


('Analyst', 3333.4166666666665, 4165.0, 3751.675987685993)
('Associate', 5846.166666666667, 8300.25, 7266.915094339623)
('Director', 17832.25, 32342.666666666668, 23914.265625)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134832)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129032)
